In [1]:
#hide
#default_exp dev.specgen

# Open-API Specification Generation

<br>

### Imports

In [2]:
#exports
import numpy as np
import pandas as pd

import os
import yaml

from jinja2 import Template

In [3]:
from IPython.display import JSON

In [4]:
#exports
def init_spec(
    title='BMRS API',
    description='API for the Elexon Balancing Mechanism Reporting Service',
    root_url='https://api.bmreports.com'
):
    API_spec = dict()

    API_spec['title'] = title
    API_spec['description'] = description
    API_spec['root_url'] = root_url
    
    return API_spec

In [5]:
API_spec = init_spec()

API_spec

{'title': 'BMRS API',
 'description': 'API for the Elexon Balancing Mechanism Reporting Service',
 'root_url': 'https://api.bmreports.com'}

In [6]:
#exports
def load_endpoints_df(endpoints_fp: str='data/endpoints.csv'):
    df_endpoints = pd.read_csv(endpoints_fp)

    date_idxs = (df_endpoints['Sample Data'].str.count('/')==2).replace(np.nan, False)
    time_idxs = df_endpoints['Sample Data'].str.contains(':').replace(np.nan, False)

    df_endpoints.loc[date_idxs & ~time_idxs, 'Sample Data'] = pd.to_datetime(df_endpoints.loc[date_idxs & ~time_idxs, 'Sample Data']).dt.strftime('%Y-%m-%d')
    df_endpoints.loc[date_idxs & time_idxs, 'Sample Data'] = pd.to_datetime(df_endpoints.loc[date_idxs & time_idxs, 'Sample Data']).dt.strftime('%Y-%m-%d %H:%M:%S')

    df_endpoints['Sample Data'] = df_endpoints['Sample Data'].fillna('')
    df_endpoints['Field Name'] = df_endpoints['Field Name'].str.replace(' ', '')
    
    return df_endpoints

In [7]:
df_endpoints = load_endpoints_df('../data/endpoints.csv')
df_endpoints.to_csv('../data/endpoints.csv', index=False)

df_endpoints.head(3)

,id,name,version,method,direction,Field Name,Field Type,Remarks,Mandatory,Format,Sample Data,tags
0,B1720,Amount Of Balancing Reserves Under Contract Se...,1,get,request,APIKey,String,NaN,Yes,NaN,AP8DA23,Balancing
1,B1720,Amount Of Balancing Reserves Under Contract Se...,1,get,request,SettlementDate,String,NaN,Yes,YYYY-MM-DD,2021-01-01,Balancing
2,B1720,Amount Of Balancing Reserves Under Contract Se...,1,get,request,Period,String,NaN,Yes,*/1-50,1,Balancing


In [8]:
#exports
def get_endpoint_single_attr(df_endpoint, attribute='version'):
    attr_val = df_endpoint[attribute].unique()
    assert len(attr_val)==1, f'Expected only 1 {attribute}, instead found {len(attr_val)}'
    attr_val = attr_val[0]

    return attr_val

In [9]:
endpoint_id = 'B1720'
df_endpoint = df_endpoints.query(f'id==@endpoint_id')

get_endpoint_single_attr(df_endpoint, 'version')

1

In [10]:
#exports
def init_stream_dict(df_endpoint, endpoint_id):
    version = get_endpoint_single_attr(df_endpoint, 'version')
    name = get_endpoint_single_attr(df_endpoint, 'name')
    tags = get_endpoint_single_attr(df_endpoint, 'tags')

    stream = dict()
    
    stream['endpoint'] = f'/BMRS/{endpoint_id}/v{version}'
    stream['x-title'] = endpoint_id
    stream['description'] = name
    stream['parameters'] = list()
    
    if isinstance(tags, str):
        stream['tags'] = tags.split(', ')
    
    return stream

In [11]:
stream = init_stream_dict(df_endpoint, endpoint_id)

stream

{'endpoint': '/BMRS/B1720/v1',
 'x-title': 'B1720',
 'description': 'Amount Of Balancing Reserves Under Contract Service',
 'parameters': [],
 'tags': ['Balancing']}

In [12]:
#exports
def add_params_to_stream_dict(
    df_endpoint: pd.DataFrame, 
    stream: dict,
    field_type_map: dict={
        'String': 'string',
        'Int': 'integer',
        'int': 'integer',
        'Integer': 'integer',
        'Date': 'string'
    }
):
    for _, (param_name, param_type, param_sample) in df_endpoint.query('direction=="request"')[['Field Name', 'Field Type', 'Sample Data']].iterrows():               
        parameter = dict()
        
        parameter['name'] = param_name
        parameter['type'] = field_type_map[param_type]
        
        if param_type in ['Date']:
            parameter['format'] = 'date'
        
        if param_name == 'APIKey':
            parameter['format'] = 'password'
        
        if param_sample == 'csv/xml':
            parameter['examples'] = {f'{param_sub_sample}': {'value': param_sub_sample} for param_sub_sample in param_sample.split('/')}
        else:
            parameter['example'] = param_sample
        
        stream['parameters'] += [parameter]
        
    return stream

In [13]:
stream = add_params_to_stream_dict(df_endpoint, stream)

stream

{'endpoint': '/BMRS/B1720/v1',
 'x-title': 'B1720',
 'description': 'Amount Of Balancing Reserves Under Contract Service',
 'parameters': [{'name': 'APIKey',
   'type': 'string',
   'format': 'password',
   'example': 'AP8DA23'},
  {'name': 'SettlementDate', 'type': 'string', 'example': '2021-01-01'},
  {'name': 'Period', 'type': 'string', 'example': '1'},
  {'name': 'ServiceType',
   'type': 'string',
   'examples': {'csv': {'value': 'csv'}, 'xml': {'value': 'xml'}}}],
 'tags': ['Balancing']}

In [14]:
#exports
def add_streams_to_spec(API_spec, df_endpoints):
    API_spec['streams'] = list()
    endpoint_ids = sorted(list(df_endpoints['id'].unique()))

    for endpoint_id in endpoint_ids:
        df_endpoint = df_endpoints.query(f'id==@endpoint_id')

        stream = init_stream_dict(df_endpoint, endpoint_id)
        stream = add_params_to_stream_dict(df_endpoint, stream)

        API_spec['streams'] += [stream]
        
    return API_spec

In [15]:
API_spec = add_streams_to_spec(API_spec, df_endpoints)
        
JSON(API_spec)

<IPython.core.display.JSON object>

In [16]:
#exports
def construct_spec(
    df_endpoints: pd.DataFrame,
    title: str='BMRS API',
    description: str='API for the Elexon Balancing Mechanism Reporting Service',
    root_url: str='https://api.bmreports.com'
):
    API_spec = init_spec()
    API_spec = add_streams_to_spec(API_spec, df_endpoints)
    
    return API_spec

In [17]:
%%time

API_spec = construct_spec(df_endpoints)

Wall time: 409 ms


In [18]:
#exports
def save_spec(
    API_spec: dict,
    in_fp: str='../templates/open_api_spec.yaml',
    out_fp: str='../data/BMRS_API.yaml'
):
    rendered_schema = Template(open(in_fp).read()).render(API_spec=API_spec)

    with open(out_fp, 'w') as f:
        try:
            f.write(rendered_schema)
        except e as exc:
            raise exc

In [19]:
save_spec(API_spec)

In [20]:
#exports
def load_API_yaml(fp='../data/BMRS_API.yaml'):
    with open(fp, 'r') as stream:
        try:
            API_yaml = yaml.safe_load(stream)
        except yaml.YAMLError as exc:
            raise exc
            
    return API_yaml

In [21]:
API_yaml = load_API_yaml(fp='../data/BMRS_API.yaml')

JSON(API_yaml)

<IPython.core.display.JSON object>

In [22]:
# https://app.swaggerhub.com/apis/AyrtonB/default-title/0.1

In [23]:
#hide
from ElexonDataPortal.dev.nbdev import notebook2script
notebook2script('02-spec-gen.ipynb')

Converted 02-spec-gen.ipynb.
